# Repository Footprint Analysis

### The repository foot print is computed for each reactor, each storage time and each number of fuel elements per package. This is based on the NWPY package by Atz (2019)

In [20]:
%matplotlib inline
import nwpyimport pandas as pd
from nwpy import fuelcycle
from nwpy.fuelcycle import stage
from nwpy.plot import TimePlot as tp
from nwpy.repository_area import repository
from nwpy.repository_area import waste
from nwpy import stream
import math
import matplotlib.pyplot as plt
import numpy as np

# Functions for NWPY

### Define the parameters and read OpenMC outputs 

In [22]:
def get_n_wf(reactor):
    """Return a list with the number of fuel unit per canister according to the technology of reactor"""
    ## ---- Parameters: 
    # reactor (str) - reactor name
    ## ---- Returns: 
    # n_wf - the number of fuel unit per canister according to the technology of reactor"""
    # ** Need to fix this at some point: Except for *PWR* and *SFR*, everything is TRISO

    if ('PWR' in reactor):
        n_wf = [1, 2, 3, 4, 12, 21]
    elif ('SFR' in reactor):
        n_wf = [1, 3, 7, 19]
    else: # Correspond to the reactors using TRISO particles
        n_wf = [6, 18, 42, 78, 114]
    return n_wf

In [23]:
def get_results(reactor,primary_path):
    """Read the cooling_reactor.csv result file from OpenMC simulation"""
    ## ---- Parameters: 
    # reactor (str) - reactor name
    # primary_path (str) - a path to the directory in which openmc files are stored
    ## ---- Returns: 
    # results - activity, decay heat as a function of time after the discharge
        
    result    = pd.read_csv(primary_path+"Results/"+ reactor+ "/cooling"+reactor+".csv", sep=',') # path to the coolingreactor.csv file
    discharge = result.iloc[1, 0]
    
    result['cooling time'] = (result.iloc[:, 0]-discharge)/365
    result.set_index('cooling time', inplace=True)
    
    return result

In [24]:
def get_canister(reactor):
    """Assign a canister design for each reactor technology"""
    ## ---- Parameters: 
    # reactor (str) - reactor name
    ## ---- Returns: 
    # canister - canister parameters
    # ** Need to fix this at some point: Except for *PWR* and *SFR*, everything is TRISO
    
    if ('PWR' in reactor):
        canister = {} # values in m
        canister[1] = {'Diameter': 0.41, 'Length': 5.0, 'Thickness':0.125}
        canister[2] = {'Diameter': 0.82, 'Length': 5.0, 'Thickness':0.125}
        canister[3] = {'Diameter': 0.82, 'Length': 5.0, 'Thickness':0.125}
        canister[4] = {'Diameter': 0.82, 'Length': 5.0, 'Thickness':0.125}
        canister[12] = {'Diameter': 1.29, 'Length': 5.0, 'Thickness':0.125}
        canister[21] = {'Diameter': 1.60, 'Length': 5.0, 'Thickness':0.125}
    elif ('SFR' in reactor):
        canister = {}
        canister[1] = {'Diameter': 0.45,'Length': 5.0, 'Thickness':0.125}
        canister[3] = {'Diameter': 0.57,'Length': 5.0, 'Thickness':0.125}# m
        canister[7] = {'Diameter': 0.78,'Length': 5.0, 'Thickness':0.125}
        canister[19] = {'Diameter': 1.11,'Length': 5.0, 'Thickness':0.125}
    else: # for reactors using TRISO particles
        canister = {}
        canister[6] = {'Diameter': 0.66+0.125,
                       'Length': 4.7625+0.125,
                       'Thickness':0.125} # m
        canister[18] = {'Diameter': 0.9298+0.125,
                        'Length': 4.7625+0.125,
                        'Thickness':0.125} # m
        canister[42] = {'Diameter': 1.1891+0.125,
                        'Length': 4.7625+0.125,
                        'Thickness':0.125} # m
        canister[60] = {'Diameter': 1.1891+0.125,
                        'Length': 4.7625+0.125,
                        'Thickness':0.125} # m
        canister[84] = {'Diameter': 1.1891+0.125,
                        'Length': 4.7625+0.125,
                        'Thickness':0.125} # m
        canister[78]={'Diameter': 1.9818+0.125, # Add a new hexagonal ring to the canister. This canister design is added to study the feasibility of large loading.
                     'Length': 4.7625+0.125,
                     'Thickness':0.125}
        canister[114]={'Diameter': 1.9818+0.125,
                     'Length': 4.7625+0.125,
                     'Thickness':0.125}
        
    return canister

In [25]:
def create_nwpy_input(reactor, n_wf):
    """Create the layout of the input data file for the Waste form instance"""
    ## ---- Parameters: 
    # reactor (str) - reactor name
    # n_wf - the number of fuel unit per canister according to the technology of reactor"""    
    ## ---- Returns: 
    # df - the input data file for the Waste form instance
    
    can = get_canister(reactor)[n_wf]
    
    pkg_diameter = can.get("Diameter")
    pkg_length   = can.get("Length")
    
    data={'name': 'uox', 'id':'snf', 'pkg_diameter': pkg_diameter, 'pkg_length': pkg_length, 'n_wf':n_wf }
    df = pd.Series(data)
    
    return df

In [27]:
def get_hm_mass(n_wf, reactor):
    """For each reactor technology and each design of canister, return the corresponding total heavy metal loading"""
    ## ---- Parameters: 
    # reactor (str) - reactor name
    # n_wf - the number of fuel unit per canister according to the technology of reactor"""    
    ## ---- Returns: 
    # hm_mass - total heavy metal loading per canister
    
    # For each kind of reactor, assign the heavy metal loading corresponding to the respective fuel unit and fuel density
    if ('PWR' in reactor):
        hm_per_asm = 460000.0 # g/asm (460 kg)        
    elif (reactor =='HTGR'):
        hm_per_asm = 8600.0 # g initial HM per prismatic block (8.6 kg)
    elif ('HPR' in reactor):
        hm_per_asm = 8600.0 # g initial HM per prismatic block (8.6 kg)
    elif ('SFR' in reactor):
        hm_per_asm=98450.0 #g/asm (98.450 kg)
    elif (reactor == "HTGR_FCM"):
        hm_per_asm = 21200.0 # g initial HM per prismatic block for FCM-HTGR
    else:
        print('no HM mass')
    hm_mass= n_wf*hm_per_asm
    return hm_mass

In [28]:
def format_decayheat(reactor, primary_path, n_wf):
    """
    Create a Serie with the decay heat results from the coolingreactor.csv output file. 
    The decay heat corresponds to the total decay heat emitted per canister. Depends on the technology of the reactor and the canister design (number of waste forms).
    """
    ## ---- Parameters: 
    # reactor (str) - reactor name
    # primary_path (str) - a path to the directory in which openmc files are stored
    # n_wf - the number of fuel unit per canister according to the technology of reactor"""        
    ## ---- Returns: 
    # save the decay heat in "path"

    result     = get_results(reactor, primary_path)
    hm_mass    = get_hm_mass(n_wf, reactor)
    decay_heat = result['Decay heat (W/g)']*hm_mass
    
    df_decayheat = create_nwpy_input(reactor, n_wf)
    df_decayheat = pd.concat([ df_decayheat, decay_heat]) # Add the decay heat data to the data file for the Waste form instance

    path = primary_path+'Results/'+reactor+'/repository_'+reactor+'_N'+str(n_wf)+ '.csv'
    df_decayheat.to_csv(path_or_buf=path, index=True, header=True)

### Create a repository and a waste form

In [29]:
def repository_instance(rock_type, reactor, primary_path, storagetime, n_wf):
    """
    Create a repository instance in the 'rock_type' evironment, with a specific storagetime before disposal, containing the nuclear waste from the 'reactor' reactor.
    The repository contains identical packages with 'nf_per_can' waste units per canister and identical surface storage storage before disposal.
    """
    ## ---- Parameters:
    # rock_type - repository rock type
    # reactor (str) - reactor name
    # primary_path (str) - a path to the directory in which openmc files are stored
    # storagetime - storage time in years
    # n_wf - the number of fuel unit per canister according to the technology of reactor"""        
    ## ---- Returns: 
    # rep - repository instance in nwpy
    
    
    path = primary_path+'Results/'+reactor+'/repository_'+reactor+'_N'+str(n_wf)+ '.csv'
    wf   = waste.Waste(path)
    
    rep = repository.Repository(rock_type, st=storagetime)
    rep.load_waste(wf)
    
    return rep

In [30]:
def get_app(rep, st_time, n_wf):
    """
    Returns a list of area per package for a specific reactor and canister design according to the surface storage time.
    A list of surface storage time is given in input
    """
    ## ---- Parameters:
    # rep - repository instance in nwpy
    # st_time - storage time in years
    # n_wf - the number of fuel unit per canister according to the technology of reactor"""        
    ## ---- Returns:
    # list_app - list of area per package for different storage time
    # PRT - list of peak rock temperature
    
    list_app = [] # list containing the area per package
    for storagetime in st_time:
        rep.update_st(storagetime) # Update the surface storage time
        app = rep.calculate_app(log=True)
        list_app.append(app)
    return list_app    

In [31]:
def get_N_pkg(reactor, primary_path, n_wf): 
    """Calculate the number of package produced per 100 GWe.year"""
    ## ---- Parameters:
    # reactor (str) - reactor name
    # primary_path (str) - a path to the directory in which openmc files are stored
    # n_wf - the number of fuel unit per canister according to the technology of reactor"""        
    ## ---- Returns:
    # list_app - list of area per package for different storage time
    # N_pkg - the number of canisters in the repository   
    
    result = get_results(reactor, primary_path)
    
    hm_mass  = get_hm_mass(n_wf, reactor)
    SNF_mass = result['SNF mass (t/GWe.y)'][0]
    
    elec_per_pkg = hm_mass*(10**-6)/SNF_mass # electricity produced per waste package 
    N_pkg        = 1/elec_per_pkg
    
    return N_pkg

### Output CSV file with app and total area

In [32]:
def output_pkg(rock_type, reactor, primary_path, st_time):
    """
    Generate an output .csv file for each number of package per canister according to the reactor technology.
    """
    ## ---- Parameters:
    # rock_type - repository rock type
    # reactor (str) - reactor name
    # primary_path (str) - a path to the directory in which openmc files are stored
    # stotime - storage time in years
    ## ---- Returns:
    # csv files containing the repository footprint products
    
    n_wf = get_n_wf(reactor)
    for i in n_wf: # iterate on the number of fuel unit per canister
        print(i)
        format_decayheat(reactor, primary_path, i) # create an output file with the decay heat data
        rep = repository_instance(rock_type, reactor, primary_path, st_time, i) # create the repository instance
        list_app = get_app(rep, st_time, i) # calculate the corresponding area per package
        N_pkg         = get_N_pkg(reactor, primary_path, i)
  
        out_rep  = pd.DataFrame(data = list_app, index=st_time, columns=['Area per package (m2)'])
        tot_area = [N_pkg*j/(10**6) for j in list_app]
    
        out_rep['Total area (km2)']       = tot_area
        out_rep['Required st (y)']        = rep.calculate_required_st()
        out_rep['#packages per GWe.y']     = N_pkg
        path=primary_path+'Results/'+reactor+'/footprint_'+ rock_type+'_'+reactor+'_'+str(i)+'.csv' # path and name of the output csv file
        out_rep.to_csv(path_or_buf=path, index=True, header=True)          

# Example

In [1]:
# List of reactor to study
reactors = ['Ref_PWR', 'SPWR', 'HTGR','HTGR_FCM', 'HPR','SFR']

# List of the surface storage time to consider
st=[3.0, 5.0, 10.0, 20.0, 50.0, 100.0]

# Type of repository
rock_type = 'clay'

In [33]:
# Create the output csv file for the comparison
for reactor in reactors:
    print(reactor)
    primary_path = './'
    output_pkg(rock_type, reactor, primary_path, st)

Ref_PWR
1


/Users/harukowainwright/anaconda3/lib/python3.11/site-packages/nwpy/repository_area/waste.py:202: RuntimeWarning: divide by zero encountered in log10
  logq[i] = np.log10(heat[i])


No solution; single package violates constraint
No solution; single package violates constraint
[10. 10.]
iter 1;   DS = 10.000  WPS = 10.000  A = 6984.620   WP Peak Temp = 88.764
[10. 10.]
iter 2;   DS = 10.000  WPS = 10.000  A = 6984.620   WP Peak Temp = 88.764
[11. 10.]
iter 3;   DS = 11.000  WPS = 10.000  A = 7664.620   WP Peak Temp = 87.306
[10. 11.]
iter 4;   DS = 10.000  WPS = 11.000  A = 7641.996   WP Peak Temp = 87.306
[9.29289322 9.29289322]
iter 5;   DS = 9.293  WPS = 9.293  A = 6070.952   WP Peak Temp = 91.656
[8.59827758 8.57351216]
iter 6;   DS = 8.598  WPS = 8.574  A = 5221.680   WP Peak Temp = 95.336
[8.05252818 7.73556371]
iter 7;   DS = 8.053  WPS = 7.736  A = 4453.988   WP Peak Temp = 99.814
[7.66142107 8.04706849]
iter 8;   DS = 7.661  WPS = 8.047  A = 4402.869   WP Peak Temp = 100.107
[8.38105949 7.35271949]
iter 9;   DS = 8.381  WPS = 7.353  A = 4417.772   WP Peak Temp = 100.097
[7.87539635 7.87539635]
iter 10;  DS = 7.875  WPS = 7.875  A = 4432.118   WP Peak Temp

/Users/harukowainwright/anaconda3/lib/python3.11/site-packages/nwpy/repository_area/waste.py:202: RuntimeWarning: divide by zero encountered in log10
  logq[i] = np.log10(heat[i])


No solution; single package violates constraint
No solution; single package violates constraint
No solution; single package violates constraint
[10. 10.]
iter 1;   DS = 10.000  WPS = 10.000  A = 7019.470   WP Peak Temp = 112.970
[10. 10.]
iter 2;   DS = 10.000  WPS = 10.000  A = 7019.470   WP Peak Temp = 112.970
[11. 10.]
iter 3;   DS = 11.000  WPS = 10.000  A = 7699.470   WP Peak Temp = 109.909
[10. 11.]
iter 4;   DS = 10.000  WPS = 11.000  A = 7680.126   WP Peak Temp = 109.909
[10.70710678 11.70710678]
iter 5;   DS = 10.707  WPS = 11.707  A = 8705.368   WP Peak Temp = 106.122
[11.41421356 12.41421356]
iter 6;   DS = 11.414  WPS = 12.414  A = 9794.610   WP Peak Temp = 102.998
[12.35789589 12.7450665 ]
iter 7;   DS = 12.358  WPS = 12.745  A = 10850.629  WP Peak Temp = 100.754
[12.09175115 13.1683477 ]
iter 8;   DS = 12.092  WPS = 13.168  A = 10959.202  WP Peak Temp = 100.544
[13.19214638 12.19368086]
iter 9;   DS = 13.192  WPS = 12.194  A = 11087.561  WP Peak Temp = 100.342
[12.8205047

...

iter 7;   DS = 5.467  WPS = 5.467  A = 4796.717   WP Peak Temp = 101.020
[5.54125136 5.54125136]
iter 8;   DS = 5.541  WPS = 5.541  A = 4920.789   WP Peak Temp = 99.997
[5.5522999  5.53020282]
iter 9;   DS = 5.552  WPS = 5.530  A = 4921.125   WP Peak Temp = 99.997
[5.54125136 5.54125136]
iter 1;   DS = 5.541  WPS = 5.541  A = 6598.730   WP Peak Temp = 105.080
[5.54125136 5.54125136]
iter 2;   DS = 5.541  WPS = 5.541  A = 6598.730   WP Peak Temp = 105.080
[6.54125136 5.54125136]
iter 3;   DS = 6.541  WPS = 5.541  A = 7754.815   WP Peak Temp = 98.251
[5.54125136 6.54125136]
iter 4;   DS = 5.541  WPS = 6.541  A = 7717.463   WP Peak Temp = 98.251
[5.91320777 5.91320777]
iter 5;   DS = 5.913  WPS = 5.913  A = 7471.980   WP Peak Temp = 99.788
[6.00159612 5.82481943]
iter 6;   DS = 6.002  WPS = 5.825  A = 7473.750   WP Peak Temp = 99.793
[5.82481943 5.82481943]
iter 7;   DS = 5.825  WPS = 5.825  A = 7259.556   WP Peak Temp = 100.982
[5.89754802 5.89754802]
iter 8;   DS = 5.898  WPS = 5.898  A

/Users/harukowainwright/anaconda3/lib/python3.11/site-packages/nwpy/repository_area/waste.py:202: RuntimeWarning: divide by zero encountered in log10
  logq[i] = np.log10(heat[i])


No solution; single package violates constraint
No solution; single package violates constraint
[10. 10.]
iter 1;   DS = 10.000  WPS = 10.000  A = 7016.070   WP Peak Temp = 126.562
[10. 10.]
iter 2;   DS = 10.000  WPS = 10.000  A = 7016.070   WP Peak Temp = 126.562
[11. 10.]
iter 3;   DS = 11.000  WPS = 10.000  A = 7696.070   WP Peak Temp = 122.863
[10. 11.]
iter 4;   DS = 10.000  WPS = 11.000  A = 7676.406   WP Peak Temp = 122.863
[10.70710678 11.70710678]
iter 5;   DS = 10.707  WPS = 11.707  A = 8701.422   WP Peak Temp = 118.286
[11.41421356 12.41421356]
iter 6;   DS = 11.414  WPS = 12.414  A = 9790.437   WP Peak Temp = 114.396
[12.02811985 13.20359247]
iter 7;   DS = 12.028  WPS = 13.204  A = 10926.464  WP Peak Temp = 111.109
[12.33796987 14.15437792]
iter 8;   DS = 12.338  WPS = 14.154  A = 11970.795  WP Peak Temp = 108.603
[13.24875686 14.56725448]
iter 9;   DS = 13.249  WPS = 14.567  A = 13190.773  WP Peak Temp = 106.278
[13.85037446 15.36603873]
iter 10;  DS = 13.850  WPS = 15.3

/Users/harukowainwright/anaconda3/lib/python3.11/site-packages/nwpy/repository_area/waste.py:202: RuntimeWarning: divide by zero encountered in log10
  logq[i] = np.log10(heat[i])


No solution; single package violates constraint
No solution; single package violates constraint
No solution; single package violates constraint
No solution; single package violates constraint
No solution; single package violates constraint
